In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [30]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, LSTM, Flatten, TimeDistributed
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Dataset path
data_path = "/kaggle/input/wrosely58"

# Verify the dataset path and list files
if not os.path.exists(data_path):
    raise FileNotFoundError(f"The path {data_path} does not exist.")

print("Files in the dataset directory:")
files = os.listdir(data_path)
print(files)

# Load and combine all CSV files into a single DataFrame
all_data = []
for file in files:
    if file.endswith(".csv"):
        file_path = os.path.join(data_path, file)
        df = pd.read_csv(file_path)
        all_data.append(df)

# Combine all data into a single DataFrame
combined_data = pd.concat(all_data, axis=0, ignore_index=True)

# Display dataset information
print("Combined data shape:", combined_data.shape)
print("Combined data columns:", combined_data.columns)

# Drop the 'timestamp' column (non-numeric)
if 'timestamp' in combined_data.columns:
    combined_data = combined_data.drop(columns=['timestamp'])

# Identify the label column
label_column = 'label'  # Update this to the correct column name for labels
if label_column not in combined_data.columns:
    raise KeyError(f"Column '{label_column}' not found in the dataset. Available columns: {combined_data.columns}")

# Separate features (X) and labels (y)
X = combined_data.drop(columns=[label_column]).values
y = combined_data[label_column].values

# Encode labels to integers
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Normalize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reshape data for CNN-LSTM input (samples, timesteps, features)
# Assuming each sample has 128 timesteps and 6 features (e.g., back_x, back_y, back_z, thigh_x, thigh_y, thigh_z)
timesteps = 128
features = X.shape[1]
X = X.reshape(-1, timesteps, features)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build CNN-LSTM model
model = Sequential()

# CNN layers
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None, timesteps, features)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Conv1D(filters=128, kernel_size=3, activation='relu')))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))

# LSTM layer
model.add(LSTM(100, return_sequences=False))
model.add(Dropout(0.5))

# Fully connected layer
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

# Output layer
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

# Save the model
model.save("cnn_lstm_har_model.h5")

Files in the dataset directory:
['train.csv', 'test.csv']
Combined data shape: (5150, 563)
Combined data columns: Index(['rn', 'activity', 'tBodyAcc.mean.X', 'tBodyAcc.mean.Y',
       'tBodyAcc.mean.Z', 'tBodyAcc.std.X', 'tBodyAcc.std.Y', 'tBodyAcc.std.Z',
       'tBodyAcc.mad.X', 'tBodyAcc.mad.Y',
       ...
       'fBodyBodyGyroJerkMag.meanFreq', 'fBodyBodyGyroJerkMag.skewness',
       'fBodyBodyGyroJerkMag.kurtosis', 'angle.tBodyAccMean.gravity',
       'angle.tBodyAccJerkMean.gravityMean', 'angle.tBodyGyroMean.gravityMean',
       'angle.tBodyGyroJerkMean.gravityMean', 'angle.X.gravityMean',
       'angle.Y.gravityMean', 'angle.Z.gravityMean'],
      dtype='object', length=563)


KeyError: "Column 'label' not found in the dataset. Available columns: Index(['rn', 'activity', 'tBodyAcc.mean.X', 'tBodyAcc.mean.Y',\n       'tBodyAcc.mean.Z', 'tBodyAcc.std.X', 'tBodyAcc.std.Y', 'tBodyAcc.std.Z',\n       'tBodyAcc.mad.X', 'tBodyAcc.mad.Y',\n       ...\n       'fBodyBodyGyroJerkMag.meanFreq', 'fBodyBodyGyroJerkMag.skewness',\n       'fBodyBodyGyroJerkMag.kurtosis', 'angle.tBodyAccMean.gravity',\n       'angle.tBodyAccJerkMean.gravityMean', 'angle.tBodyGyroMean.gravityMean',\n       'angle.tBodyGyroJerkMean.gravityMean', 'angle.X.gravityMean',\n       'angle.Y.gravityMean', 'angle.Z.gravityMean'],\n      dtype='object', length=563)"